In [133]:
import pandas as pd
import sqlite3
import numpy as np

In [155]:
sel_start_dat='04/09/2020 08:00:00'
sel_end_dat='04/09/2020 09:00:00'
sel_zq=60
sql_path=r"D:\NR干扰分析\nrgr.db"
conn = sqlite3.connect(sql_path)
cursor = conn.cursor()
cursor.execute("""select ta.*,tb.* from nr_rb1_interfence ta,nr_rb2_interfence tb
                  where ta.时间=tb.时间 and ta.小区名称=tb.小区名称 and ta.时间 between '%(dat1)s' and '%(dat2)s'
                  and ta.`周期`='%(z1)d' and ta.`周期`=tb.`周期`;
               """ %dict(dat1=sel_start_dat,dat2=sel_end_dat,z1=sel_zq))
cols = cursor.description
columns_list = [col[0] for col in cols]
result=cursor.fetchall()
cursor.close()
conn.close()
if len(result)>0:
    data_list=[]
    for item in result:
        rb_nums=list(item)
        [rb_nums.pop(152) for i in range(4)]
        data_list.append(rb_nums)
    [columns_list.pop(152) for i in range(4)]
    df=pd.DataFrame(data_list)
print(df.head())
df.columns

                   0    1                   2                        3    \
0  04/09/2020 08:00:00   60      D_汉阳湘隆时代广场-H5H      D_汉阳湘隆时代广场-H5H-2612   
1  04/09/2020 08:00:00   60  D_蔡甸珠山湖大道与全力三路-H5H  D_蔡甸珠山湖大道与全力三路-H5H-2611   
2  04/09/2020 08:00:00   60       D_民航小区路7号-H5H       D_民航小区路7号-H5H-2612   
3  04/09/2020 08:00:00   60       D_武汉生物学院4-H5H          D_武汉生物学院4-H5H-2   
4  04/09/2020 08:00:00   60      D_汉阳湘隆时代广场-H5H      D_汉阳湘隆时代广场-H5H-2611   

    4     5     6     7     8     9    ...   271   272   273   274   275  \
0  -113  -113  -112  -112  -113  -113  ...  -115  -115  -115  -115  -115   
1  -114  -113  -113  -113  -113  -113  ...  -115  -115  -115  -115  -115   
2  -113  -112  -112  -112  -112  -112  ...  -115  -115  -115  -115  -115   
3  -114  -114  -114  -114  -114  -114  ...  -114  -114  -114  -114  -114   
4  -110  -111  -111  -110  -110  -111  ...  -114  -114  -114  -114  -114   

    276   277   278   279   280  
0  -115  -114  -112  -114  -114  
1  -115  -114  -10

RangeIndex(start=0, stop=281, step=1)

In [257]:
df_avg_res=pd.DataFrame()
for i in range(277):
    df=df[~df[i+4].isin(["NIL"])]
    df[i+4]=df[i+4].astype(float)
    df_avg = df.groupby(by=[3])[i+4].mean()
    df_avg=df_avg.reset_index()
    if len(df_avg_res)==0:
        print(i+4)
        df_avg_res=df_avg
    else:
        df_avg_res=pd.merge(df_avg_res,df_avg,how='left',on=3)
df_avg_res.head(0)

4


,3,4,5,6,7,8,9,10,11,12,...,271,272,273,274,275,276,277,278,279,280


In [274]:
df_date=df.copy()
df_date['日期']=df_date.apply(lambda x:x[0].split(' ')[0].split(r'/')[2]+x[0].split(' ')[0].split(r'/')[0]+x[0].split(' ')[0].split(r'/')[1],axis=1)


In [275]:
df_date_avg_res=pd.DataFrame()
for i in range(277):
    df_date=df_date[~df_date[i+4].isin(["NIL"])]
    df_date[i+4]=df_date[i+4].astype(float)
    df_date_avg = df_date.groupby(by=['日期',3])[i+4].mean()
    df_date_avg=df_date_avg.reset_index()
    if len(df_date_avg_res)==0:
        df_date_avg_res=df_date_avg
        df_date_avg_res.rename(columns={0:'日期'},inplace= True)
    else:
        df_date_avg_res=pd.merge(df_date_avg_res,df_date_avg,how='left',on=['日期',3])
df_date_avg_res

,日期,3,4,5,6,7,8,9,10,11,...,271,272,273,274,275,276,277,278,279,280
0,20200409,D_ 汉阳归元寺停车场-H5H-1,-112.0,-112.0,-113.0,-113.0,-113.0,-113.0,-113.0,-113.0,...,-114.5,-115.0,-115.0,-115.0,-114.5,-115.0,-114.0,-112.5,-112.0,-113.0
1,20200409,D_ 汉阳归元寺停车场-H5H-2,-111.0,-111.0,-113.0,-113.0,-113.0,-113.0,-113.0,-113.0,...,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-114.0,-112.0,-112.0,-112.5
2,20200409,D_ 汉阳归元寺停车场-H5H-3,-114.0,-114.0,-113.0,-113.0,-113.0,-113.0,-113.0,-113.0,...,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-114.0,-114.0,-114.0,-113.5
3,20200409,D_1886汽车大道台北路停车场西-H5H-2611,-110.5,-110.5,-113.0,-113.0,-113.0,-113.0,-113.0,-113.0,...,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-114.0,-113.0,-113.0,-113.0
4,20200409,D_1886汽车大道台北路停车场西-H5H-2612,-113.5,-113.5,-112.5,-112.5,-112.5,-112.5,-112.5,-112.5,...,-114.0,-114.0,-114.0,-114.0,-114.0,-114.0,-114.0,-113.0,-114.0,-114.0
5,20200409,D_1886汽车大道台北路停车场西-H5H-2613,-113.0,-113.0,-112.5,-112.5,-112.5,-112.5,-112.5,-112.5,...,-115.0,-115.0,-115.0,-115.0,-115.0,-115.0,-114.0,-112.5,-114.0,-113.0
6,20200409,D_CBD管委会-H5H-2611,-107.0,-107.0,-110.0,-109.5,-110.0,-110.0,-110.0,-110.0,...,-114.0,-114.0,-114.0,-114.0,-114.0,-114.0,-114.0,-112.0,-109.0,-109.5
7,20200409,D_CBD管委会-H5H-2612,-111.5,-111.5,-110.5,-110.5,-110.5,-110.5,-110.5,-110.5,...,-114.0,-114.0,-114.0,-114.0,-114.0,-114.0,-114.0,-111.0,-112.0,-112.0
8,20200409,D_CBD管委会-H5H-2613,-108.5,-108.5,-112.0,-111.5,-111.5,-112.0,-112.0,-111.5,...,-114.0,-114.0,-114.0,-114.0,-114.0,-114.0,-114.0,-112.0,-110.5,-110.0
9,20200409,D_CBD网球场旁边-H5H-1,-104.5,-104.5,-105.0,-105.0,-105.5,-105.5,-105.0,-105.5,...,-113.0,-113.0,-113.0,-113.5,-113.0,-113.0,-114.0,-110.5,-106.0,-107.0


In [265]:
df_max=df_date.sort_values([4], ascending=False).groupby([3], as_index=False).first()
df_max.head()
df_max.loc[:,'干扰RB个数']=df_max.apply(lambda x:len([val for val in [x[i+5] for i in range(273)] if val>-107]),axis=1)
df_max.loc[:,'干扰RB位置']=df_max.apply(lambda x:'_'.join([str(int(val)) for val in [i+5 for i in range(273)] if x[val]>-107]) if len([val for val in [x[i+5] for i in range(273)] if val>-107])>0 else '',axis=1)
df_max

,3,0,1,2,4,5,6,7,8,9,...,274,275,276,277,278,279,280,日期,干扰RB个数,干扰RB位置
0,D_ 汉阳归元寺停车场-H5H-1,04/09/2020 09:00:00,60,D_汉阳归元寺停车场-H5H,-112.0,-112.0,-113.0,-113.0,-113.0,-113.0,...,-115.0,-114.0,-115.0,-114.0,-113.0,-112.0,-113.0,20200409,0,
1,D_ 汉阳归元寺停车场-H5H-2,04/09/2020 08:00:00,60,D_汉阳归元寺停车场-H5H,-111.0,-111.0,-113.0,-113.0,-113.0,-113.0,...,-115.0,-115.0,-115.0,-114.0,-112.0,-112.0,-113.0,20200409,0,
2,D_ 汉阳归元寺停车场-H5H-3,04/09/2020 09:00:00,60,D_汉阳归元寺停车场-H5H,-114.0,-114.0,-113.0,-113.0,-113.0,-113.0,...,-115.0,-115.0,-115.0,-114.0,-114.0,-114.0,-113.0,20200409,0,
3,D_1886汽车大道台北路停车场西-H5H-2611,04/09/2020 08:00:00,60,D_1886汽车大道台北路停车场西-H5H,-110.0,-110.0,-113.0,-113.0,-113.0,-113.0,...,-115.0,-115.0,-115.0,-114.0,-113.0,-113.0,-113.0,20200409,0,
4,D_1886汽车大道台北路停车场西-H5H-2612,04/09/2020 08:00:00,60,D_1886汽车大道台北路停车场西-H5H,-113.0,-113.0,-112.0,-112.0,-112.0,-112.0,...,-114.0,-114.0,-114.0,-114.0,-113.0,-114.0,-114.0,20200409,0,
5,D_1886汽车大道台北路停车场西-H5H-2613,04/09/2020 09:00:00,60,D_1886汽车大道台北路停车场西-H5H,-113.0,-113.0,-112.0,-112.0,-112.0,-112.0,...,-115.0,-115.0,-115.0,-114.0,-113.0,-114.0,-113.0,20200409,0,
6,D_CBD管委会-H5H-2611,04/09/2020 09:00:00,60,D_CBD管委会-H5H,-107.0,-107.0,-110.0,-110.0,-110.0,-110.0,...,-114.0,-114.0,-114.0,-114.0,-113.0,-109.0,-110.0,20200409,8,169_170_171_172_221_222_223_224
7,D_CBD管委会-H5H-2612,04/09/2020 08:00:00,60,D_CBD管委会-H5H,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,...,-114.0,-114.0,-114.0,-114.0,-110.0,-112.0,-112.0,20200409,4,225_226_227_228
8,D_CBD管委会-H5H-2613,04/09/2020 09:00:00,60,D_CBD管委会-H5H,-108.0,-108.0,-112.0,-112.0,-112.0,-112.0,...,-114.0,-114.0,-114.0,-114.0,-113.0,-110.0,-109.0,20200409,8,169_170_171_172_221_222_223_224
9,D_CBD网球场旁边-H5H-1,04/09/2020 08:00:00,60,D_CBD网球场旁边-H5H,-104.0,-104.0,-105.0,-105.0,-105.0,-105.0,...,-113.0,-113.0,-113.0,-114.0,-110.0,-105.0,-106.0,20200409,32,5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_...


In [204]:
date_set=set(df_date['日期'].to_list())
date_set

{'20200409'}

In [224]:
d4_cols=[i+5 for i in range(51)]
d5_cols=[i+56 for i in range(55)]
d6_cols=[i+111 for i in range(56)]
d1_cols=[i+167 for i in range(56)]
d2_cols=[i+223 for i in range(54)]
print(d4_cols)

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]


In [249]:
df_avg_res.iloc[:5,270:]

,273,274,275,276,277,278,279,280
0,-115.0,-115.0,-114.5,-115.0,-114.0,-112.5,-112.0,-113.0
1,-115.0,-115.0,-115.0,-115.0,-114.0,-112.0,-112.0,-112.5
2,-115.0,-115.0,-115.0,-115.0,-114.0,-114.0,-114.0,-113.5
3,-115.0,-115.0,-115.0,-115.0,-114.0,-113.0,-113.0,-113.0
4,-114.0,-114.0,-114.0,-114.0,-114.0,-113.0,-114.0,-114.0


In [253]:
df_avg_res.loc[:,'全时段D4干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d4_cols]),axis=1)
df_avg_res.loc[:,'全时段D5干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d5_cols]),axis=1)
df_avg_res.loc[:,'全时段D6干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d6_cols]),axis=1)
df_avg_res.loc[:,'全时段D1干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d1_cols]),axis=1)
df_avg_res.loc[:,'全时段D2干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d2_cols]),axis=1)
totle_res=df_avg_res[[3,4,'全时段D4干扰均值','全时段D5干扰均值','全时段D6干扰均值','全时段D1干扰均值','全时段D2干扰均值',278,279,280]]
totle_res.rename(columns={3:'小区名称',4:'全时段全频段干扰均值',278:'全时段Gp干扰均值',279:'全时段U_s6干扰均值',280:'全时段U_s6干扰均值'},inplace=True)
totle_res

,小区名称,全时段全频段干扰均值,全时段D4干扰均值,全时段D5干扰均值,全时段D6干扰均值,全时段D1干扰均值,全时段D2干扰均值,全时段Gp干扰均值,全时段U_s6干扰均值,全时段U_s6干扰均值
0,D_ 汉阳归元寺停车场-H5H-1,-112.0,-112.833333,-111.590909,-114.383929,-114.580357,-113.981481,-112.5,-112.0,-113.0
1,D_ 汉阳归元寺停车场-H5H-2,-111.0,-111.911765,-110.727273,-112.830357,-114.383929,-114.370370,-112.0,-112.0,-112.5
2,D_ 汉阳归元寺停车场-H5H-3,-114.0,-113.431373,-113.418182,-115.035714,-114.678571,-114.777778,-114.0,-114.0,-113.5
3,D_1886汽车大道台北路停车场西-H5H-2611,-110.5,-113.519608,-114.554545,-114.571429,-113.107143,-113.953704,-113.0,-113.0,-113.0
4,D_1886汽车大道台北路停车场西-H5H-2612,-113.5,-113.323529,-114.072727,-114.544643,-113.892857,-113.851852,-113.0,-114.0,-114.0
5,D_1886汽车大道台北路停车场西-H5H-2613,-113.0,-113.382353,-114.609091,-114.982143,-114.491071,-114.712963,-112.5,-114.0,-113.0
6,D_CBD管委会-H5H-2611,-107.0,-111.990196,-114.036364,-114.455357,-111.812500,-113.148148,-112.0,-109.0,-109.5
7,D_CBD管委会-H5H-2612,-111.5,-112.264706,-113.990909,-114.580357,-113.080357,-113.046296,-111.0,-112.0,-112.0
8,D_CBD管委会-H5H-2613,-108.5,-112.950980,-114.663636,-114.696429,-112.357143,-113.351852,-112.0,-110.5,-110.0
9,D_CBD网球场旁边-H5H-1,-104.5,-109.421569,-114.100000,-114.428571,-111.026786,-111.731481,-110.5,-106.0,-107.0


In [268]:
dict_post={'D4':'最强时段D4干扰均值','D5':'最强时段D5干扰均值','D6':'最强时段D6干扰均值','D1':'最强时段D1干扰均值','D2':'最强时段D2干扰均值'}
df_max.loc[:,'最强时段D4干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d4_cols]),axis=1)
df_max.loc[:,'最强时段D5干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d5_cols]),axis=1)
df_max.loc[:,'最强时段D6干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d6_cols]),axis=1)
df_max.loc[:,'最强时段D1干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d1_cols]),axis=1)
df_max.loc[:,'最强时段D2干扰均值']=df_avg_res.apply(lambda x:np.mean([x[i] for i in d2_cols]),axis=1)
df_max_res=df_max[[0,3,4,'最强时段D4干扰均值','最强时段D5干扰均值','最强时段D6干扰均值','最强时段D1干扰均值','最强时段D2干扰均值',278,279,280,'干扰RB个数','干扰RB位置']]
df_max_res.rename(columns={0:'最强时段',3:'小区名称',4:'最强时段全频段干扰均值',278:'最强时段Gp干扰均值',279:'最强时段U_s6干扰均值',280:'最强时段U_s6干扰均值'},inplace=True)
df_max_res.loc[:,'干扰分段位置']=df_max_res.apply(lambda x:[i for i in ['D4','D5','D6','D1','D2'] if x[dict_post[i]]>-107],axis=1)
df_max_res

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,最强时段,小区名称,最强时段全频段干扰均值,最强时段D4干扰均值,最强时段D5干扰均值,最强时段D6干扰均值,最强时段D1干扰均值,最强时段D2干扰均值,最强时段Gp干扰均值,最强时段U_s6干扰均值,最强时段U_s6干扰均值,干扰RB个数,干扰RB位置,干扰分段位置
0,04/09/2020 09:00:00,D_ 汉阳归元寺停车场-H5H-1,-112.0,-112.833333,-111.590909,-114.383929,-114.580357,-113.981481,-113.0,-112.0,-113.0,0,,[]
1,04/09/2020 08:00:00,D_ 汉阳归元寺停车场-H5H-2,-111.0,-111.911765,-110.727273,-112.830357,-114.383929,-114.370370,-112.0,-112.0,-113.0,0,,[]
2,04/09/2020 09:00:00,D_ 汉阳归元寺停车场-H5H-3,-114.0,-113.431373,-113.418182,-115.035714,-114.678571,-114.777778,-114.0,-114.0,-113.0,0,,[]
3,04/09/2020 08:00:00,D_1886汽车大道台北路停车场西-H5H-2611,-110.0,-113.519608,-114.554545,-114.571429,-113.107143,-113.953704,-113.0,-113.0,-113.0,0,,[]
4,04/09/2020 08:00:00,D_1886汽车大道台北路停车场西-H5H-2612,-113.0,-113.323529,-114.072727,-114.544643,-113.892857,-113.851852,-113.0,-114.0,-114.0,0,,[]
5,04/09/2020 09:00:00,D_1886汽车大道台北路停车场西-H5H-2613,-113.0,-113.382353,-114.609091,-114.982143,-114.491071,-114.712963,-113.0,-114.0,-113.0,0,,[]
6,04/09/2020 09:00:00,D_CBD管委会-H5H-2611,-107.0,-111.990196,-114.036364,-114.455357,-111.812500,-113.148148,-113.0,-109.0,-110.0,8,169_170_171_172_221_222_223_224,[]
7,04/09/2020 08:00:00,D_CBD管委会-H5H-2612,-111.0,-112.264706,-113.990909,-114.580357,-113.080357,-113.046296,-110.0,-112.0,-112.0,4,225_226_227_228,[]
8,04/09/2020 09:00:00,D_CBD管委会-H5H-2613,-108.0,-112.950980,-114.663636,-114.696429,-112.357143,-113.351852,-113.0,-110.0,-109.0,8,169_170_171_172_221_222_223_224,[]
9,04/09/2020 08:00:00,D_CBD网球场旁边-H5H-1,-104.0,-109.421569,-114.100000,-114.428571,-111.026786,-111.731481,-110.0,-105.0,-106.0,32,5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_...,[]


In [269]:
totle_res=pd.merge(totle_res,df_max_res,how='inner',on='小区名称')
totle_res

,小区名称,全时段全频段干扰均值,全时段D4干扰均值,全时段D5干扰均值,全时段D6干扰均值,全时段D1干扰均值,全时段D2干扰均值,全时段Gp干扰均值,全时段U_s6干扰均值,全时段U_s6干扰均值,...,最强时段D5干扰均值,最强时段D6干扰均值,最强时段D1干扰均值,最强时段D2干扰均值,最强时段Gp干扰均值,最强时段U_s6干扰均值,最强时段U_s6干扰均值,干扰RB个数,干扰RB位置,干扰分段位置
0,D_ 汉阳归元寺停车场-H5H-1,-112.0,-112.833333,-111.590909,-114.383929,-114.580357,-113.981481,-112.5,-112.0,-113.0,...,-111.590909,-114.383929,-114.580357,-113.981481,-113.0,-112.0,-113.0,0,,[]
1,D_ 汉阳归元寺停车场-H5H-2,-111.0,-111.911765,-110.727273,-112.830357,-114.383929,-114.370370,-112.0,-112.0,-112.5,...,-110.727273,-112.830357,-114.383929,-114.370370,-112.0,-112.0,-113.0,0,,[]
2,D_ 汉阳归元寺停车场-H5H-3,-114.0,-113.431373,-113.418182,-115.035714,-114.678571,-114.777778,-114.0,-114.0,-113.5,...,-113.418182,-115.035714,-114.678571,-114.777778,-114.0,-114.0,-113.0,0,,[]
3,D_1886汽车大道台北路停车场西-H5H-2611,-110.5,-113.519608,-114.554545,-114.571429,-113.107143,-113.953704,-113.0,-113.0,-113.0,...,-114.554545,-114.571429,-113.107143,-113.953704,-113.0,-113.0,-113.0,0,,[]
4,D_1886汽车大道台北路停车场西-H5H-2612,-113.5,-113.323529,-114.072727,-114.544643,-113.892857,-113.851852,-113.0,-114.0,-114.0,...,-114.072727,-114.544643,-113.892857,-113.851852,-113.0,-114.0,-114.0,0,,[]
5,D_1886汽车大道台北路停车场西-H5H-2613,-113.0,-113.382353,-114.609091,-114.982143,-114.491071,-114.712963,-112.5,-114.0,-113.0,...,-114.609091,-114.982143,-114.491071,-114.712963,-113.0,-114.0,-113.0,0,,[]
6,D_CBD管委会-H5H-2611,-107.0,-111.990196,-114.036364,-114.455357,-111.812500,-113.148148,-112.0,-109.0,-109.5,...,-114.036364,-114.455357,-111.812500,-113.148148,-113.0,-109.0,-110.0,8,169_170_171_172_221_222_223_224,[]
7,D_CBD管委会-H5H-2612,-111.5,-112.264706,-113.990909,-114.580357,-113.080357,-113.046296,-111.0,-112.0,-112.0,...,-113.990909,-114.580357,-113.080357,-113.046296,-110.0,-112.0,-112.0,4,225_226_227_228,[]
8,D_CBD管委会-H5H-2613,-108.5,-112.950980,-114.663636,-114.696429,-112.357143,-113.351852,-112.0,-110.5,-110.0,...,-114.663636,-114.696429,-112.357143,-113.351852,-113.0,-110.0,-109.0,8,169_170_171_172_221_222_223_224,[]
9,D_CBD网球场旁边-H5H-1,-104.5,-109.421569,-114.100000,-114.428571,-111.026786,-111.731481,-110.5,-106.0,-107.0,...,-114.100000,-114.428571,-111.026786,-111.731481,-110.0,-105.0,-106.0,32,5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_...,[]


In [270]:
totle_res.loc[:,'是否干扰小区']=totle_res.apply(lambda x:'是' if x['最强时段全频段干扰均值']>-107 else '否',axis=1)

In [271]:
totle_res

,小区名称,全时段全频段干扰均值,全时段D4干扰均值,全时段D5干扰均值,全时段D6干扰均值,全时段D1干扰均值,全时段D2干扰均值,全时段Gp干扰均值,全时段U_s6干扰均值,全时段U_s6干扰均值,...,最强时段D6干扰均值,最强时段D1干扰均值,最强时段D2干扰均值,最强时段Gp干扰均值,最强时段U_s6干扰均值,最强时段U_s6干扰均值,干扰RB个数,干扰RB位置,干扰分段位置,是否干扰小区
0,D_ 汉阳归元寺停车场-H5H-1,-112.0,-112.833333,-111.590909,-114.383929,-114.580357,-113.981481,-112.5,-112.0,-113.0,...,-114.383929,-114.580357,-113.981481,-113.0,-112.0,-113.0,0,,[],否
1,D_ 汉阳归元寺停车场-H5H-2,-111.0,-111.911765,-110.727273,-112.830357,-114.383929,-114.370370,-112.0,-112.0,-112.5,...,-112.830357,-114.383929,-114.370370,-112.0,-112.0,-113.0,0,,[],否
2,D_ 汉阳归元寺停车场-H5H-3,-114.0,-113.431373,-113.418182,-115.035714,-114.678571,-114.777778,-114.0,-114.0,-113.5,...,-115.035714,-114.678571,-114.777778,-114.0,-114.0,-113.0,0,,[],否
3,D_1886汽车大道台北路停车场西-H5H-2611,-110.5,-113.519608,-114.554545,-114.571429,-113.107143,-113.953704,-113.0,-113.0,-113.0,...,-114.571429,-113.107143,-113.953704,-113.0,-113.0,-113.0,0,,[],否
4,D_1886汽车大道台北路停车场西-H5H-2612,-113.5,-113.323529,-114.072727,-114.544643,-113.892857,-113.851852,-113.0,-114.0,-114.0,...,-114.544643,-113.892857,-113.851852,-113.0,-114.0,-114.0,0,,[],否
5,D_1886汽车大道台北路停车场西-H5H-2613,-113.0,-113.382353,-114.609091,-114.982143,-114.491071,-114.712963,-112.5,-114.0,-113.0,...,-114.982143,-114.491071,-114.712963,-113.0,-114.0,-113.0,0,,[],否
6,D_CBD管委会-H5H-2611,-107.0,-111.990196,-114.036364,-114.455357,-111.812500,-113.148148,-112.0,-109.0,-109.5,...,-114.455357,-111.812500,-113.148148,-113.0,-109.0,-110.0,8,169_170_171_172_221_222_223_224,[],否
7,D_CBD管委会-H5H-2612,-111.5,-112.264706,-113.990909,-114.580357,-113.080357,-113.046296,-111.0,-112.0,-112.0,...,-114.580357,-113.080357,-113.046296,-110.0,-112.0,-112.0,4,225_226_227_228,[],否
8,D_CBD管委会-H5H-2613,-108.5,-112.950980,-114.663636,-114.696429,-112.357143,-113.351852,-112.0,-110.5,-110.0,...,-114.696429,-112.357143,-113.351852,-113.0,-110.0,-109.0,8,169_170_171_172_221_222_223_224,[],否
9,D_CBD网球场旁边-H5H-1,-104.5,-109.421569,-114.100000,-114.428571,-111.026786,-111.731481,-110.5,-106.0,-107.0,...,-114.428571,-111.026786,-111.731481,-110.0,-105.0,-106.0,32,5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_...,[],是


In [272]:
df_date_avg_res.head()

,日期,3,4,5,6
0,20200409,D_ 汉阳归元寺停车场-H5H-1,-112.0,-112.0,-113.0
1,20200409,D_ 汉阳归元寺停车场-H5H-2,-111.0,-111.0,-113.0
2,20200409,D_ 汉阳归元寺停车场-H5H-3,-114.0,-114.0,-113.0
3,20200409,D_1886汽车大道台北路停车场西-H5H-2611,-110.5,-110.5,-113.0
4,20200409,D_1886汽车大道台北路停车场西-H5H-2612,-113.5,-113.5,-112.5


In [279]:
df_date_avg_res.loc[:,'全时段D4干扰均值']=df_date_avg_res.apply(lambda x:np.mean([x[i] for i in d4_cols]),axis=1)
df_date_avg_res.loc[:,'全时段D5干扰均值']=df_date_avg_res.apply(lambda x:np.mean([x[i] for i in d5_cols]),axis=1)
df_date_avg_res.loc[:,'全时段D6干扰均值']=df_date_avg_res.apply(lambda x:np.mean([x[i] for i in d6_cols]),axis=1)
df_date_avg_res.loc[:,'全时段D1干扰均值']=df_date_avg_res.apply(lambda x:np.mean([x[i] for i in d1_cols]),axis=1)
df_date_avg_res.loc[:,'全时段D2干扰均值']=df_date_avg_res.apply(lambda x:np.mean([x[i] for i in d2_cols]),axis=1)
totle_date_res=df_date_avg_res[['日期',3,4,'全时段D4干扰均值','全时段D5干扰均值','全时段D6干扰均值','全时段D1干扰均值','全时段D2干扰均值',278,279,280]]
totle_date_res.rename(columns={3:'小区名称',4:'全时段全频段干扰均值',278:'全时段Gp干扰均值',279:'全时段U_s6干扰均值',280:'全时段U_s11干扰均值'},inplace=True)
df_date_max=df_date.sort_values([4], ascending=False).groupby(['日期',3], as_index=False).first()
df_date_max.loc[:,'干扰RB个数']=df_date_max.apply(lambda x:len([val for val in [x[i+5] for i in range(273)] if val>-107]),axis=1)
df_date_max.loc[:,'干扰RB位置']=df_date_max.apply(lambda x:'_'.join([str(int(val)) for val in [i+5 for i in range(273)] if x[val]>-107]) if len([val for val in [x[i+5] for i in range(273)] if val>-107])>0 else '',axis=1)
df_date_max.loc[:,'最强时段D4干扰均值']=df_date_max.apply(lambda x:np.mean([x[i] for i in d4_cols]),axis=1)
df_date_max.loc[:,'最强时段D5干扰均值']=df_date_max.apply(lambda x:np.mean([x[i] for i in d5_cols]),axis=1)
df_date_max.loc[:,'最强时段D6干扰均值']=df_date_max.apply(lambda x:np.mean([x[i] for i in d6_cols]),axis=1)
df_date_max.loc[:,'最强时段D1干扰均值']=df_date_max.apply(lambda x:np.mean([x[i] for i in d1_cols]),axis=1)
df_date_max.loc[:,'最强时段D2干扰均值']=df_date_max.apply(lambda x:np.mean([x[i] for i in d2_cols]),axis=1)
df_date_max_res=df_date_max[['日期',0,3,4,'最强时段D4干扰均值','最强时段D5干扰均值','最强时段D6干扰均值','最强时段D1干扰均值','最强时段D2干扰均值',278,279,280,'干扰RB个数','干扰RB位置']]
df_date_max_res.rename(columns={0:'最强时段',3:'小区名称',4:'最强时段全频段干扰均值',278:'最强时段Gp干扰均值',279:'最强时段U_s6干扰均值',280:'最强时段U_s11干扰均值'},inplace=True)
df_date_max_res.loc[:,'干扰分段位置']=df_date_max_res.apply(lambda x:[i for i in ['D4','D5','D6','D1','D2'] if x[dict_post[i]]>-107],axis=1)


D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [280]:
totle_date_res=pd.merge(totle_date_res,df_date_max_res,how='inner',on=['日期','小区名称'])
totle_date_res

,日期,小区名称,全时段全频段干扰均值,全时段D4干扰均值,全时段D5干扰均值,全时段D6干扰均值,全时段D1干扰均值,全时段D2干扰均值,全时段Gp干扰均值,全时段U_s6干扰均值,...,最强时段D5干扰均值,最强时段D6干扰均值,最强时段D1干扰均值,最强时段D2干扰均值,最强时段Gp干扰均值,最强时段U_s6干扰均值,最强时段U_s6干扰均值,干扰RB个数,干扰RB位置,干扰分段位置
0,20200409,D_ 汉阳归元寺停车场-H5H-1,-112.0,-112.833333,-111.590909,-114.383929,-114.580357,-113.981481,-112.5,-112.0,...,-111.654545,-114.410714,-114.625000,-113.888889,-113.0,-112.0,-113.0,0,,[]
1,20200409,D_ 汉阳归元寺停车场-H5H-2,-111.0,-111.911765,-110.727273,-112.830357,-114.383929,-114.370370,-112.0,-112.0,...,-110.909091,-112.946429,-114.482143,-114.425926,-112.0,-112.0,-113.0,0,,[]
2,20200409,D_ 汉阳归元寺停车场-H5H-3,-114.0,-113.431373,-113.418182,-115.035714,-114.678571,-114.777778,-114.0,-114.0,...,-113.490909,-115.035714,-114.750000,-114.777778,-114.0,-114.0,-113.0,0,,[]
3,20200409,D_1886汽车大道台北路停车场西-H5H-2611,-110.5,-113.519608,-114.554545,-114.571429,-113.107143,-113.953704,-113.0,-113.0,...,-114.854545,-114.678571,-113.250000,-114.074074,-113.0,-113.0,-113.0,0,,[]
4,20200409,D_1886汽车大道台北路停车场西-H5H-2612,-113.5,-113.323529,-114.072727,-114.544643,-113.892857,-113.851852,-113.0,-114.0,...,-114.036364,-114.535714,-113.875000,-113.851852,-113.0,-114.0,-114.0,0,,[]
5,20200409,D_1886汽车大道台北路停车场西-H5H-2613,-113.0,-113.382353,-114.609091,-114.982143,-114.491071,-114.712963,-112.5,-114.0,...,-114.509091,-114.982143,-114.428571,-114.703704,-113.0,-114.0,-113.0,0,,[]
6,20200409,D_CBD管委会-H5H-2611,-107.0,-111.990196,-114.036364,-114.455357,-111.812500,-113.148148,-112.0,-109.0,...,-114.036364,-114.464286,-111.642857,-113.166667,-113.0,-109.0,-110.0,8,169_170_171_172_221_222_223_224,[]
7,20200409,D_CBD管委会-H5H-2612,-111.5,-112.264706,-113.990909,-114.580357,-113.080357,-113.046296,-111.0,-112.0,...,-113.981818,-114.250000,-113.017857,-112.981481,-110.0,-112.0,-112.0,4,225_226_227_228,[]
8,20200409,D_CBD管委会-H5H-2613,-108.5,-112.950980,-114.663636,-114.696429,-112.357143,-113.351852,-112.0,-110.5,...,-114.600000,-114.714286,-111.910714,-113.314815,-113.0,-110.0,-109.0,8,169_170_171_172_221_222_223_224,[]
9,20200409,D_CBD网球场旁边-H5H-1,-104.5,-109.421569,-114.100000,-114.428571,-111.026786,-111.731481,-110.5,-106.0,...,-114.127273,-114.446429,-110.910714,-111.462963,-110.0,-105.0,-106.0,32,5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_...,[]
